# Kepler Framework, Emulate v12

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [10]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [11]:
tpath_data17 = 'output/emulation/data17_table_v12.h5'
tpath_data18 = 'output/emulation/data18_table_v12.h5'
tpath_mc16   = 'output/emulation/mc16_table_v12.h5'

## Setup Chains:

In [12]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v12_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v12_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v12_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v12" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v12_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v12_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v12_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v12_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v12_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [13]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [14]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.47s/it]


In [15]:
data17_df.shape

(43311283, 111)

### Validate:

In [16]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v12.root')

2022-03-22 18:18:45,013 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v12.root


In [17]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.43s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5262  | 29972876/32047563 |
|   L2   | 93.2890  | 29896843/32047563 |
| EFCalo | 91.9757  | 29475986/32047563 |
|  HLT   | 90.0275  | 28851615/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.34s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2246  | 21450792/22765602 |
|   L2   | 94.1117  | 21425102/22765602 |
| EFCalo | 93.1869  | 21214562/22765602 |
|  HLT   | 87.0220  | 19811072/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.3531  | 1331582/1367786 |
|   L2   | 97.2642  | 1330366/1367786 |
| EFCalo | 95.1735  | 1301770/1367786 |
|  HLT   | 91.9016  | 1257017/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.35it/s]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.9865  | 59322/60541  |
|   L2   | 97.9204  | 59282/60541  |
| EFCalo | 96.6007  | 58483/60541  |
|  HLT   | 94.6466  | 57300/60541  |
+--------+----------+--------------+


In [18]:
eff_data17_Zee.save()

In [19]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v12.root')

2022-03-22 18:22:50,131 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v12.root


In [20]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v12'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.37s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  2.1368  | 221154/10349992  |
|   L2   |  2.0697  | 214216/10349992  |
| EFCalo |  1.8745  | 194009/10349992  |
|  HLT   |  0.1644  |  17018/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.90s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.8239  |  76244/9254300  |
|   L2   |  0.7744  |  71663/9254300  |
| EFCalo |  0.6304  |  58338/9254300  |
|  HLT   |  0.0017  |   160/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.8132  |  63857/7852181  |
|   L2   |  0.7556  |  59332/7852181  |
| EFCalo |  0.5819  |  45689/7852181  |
|  HLT   |  0.0024  |   189/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.47s/it]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  1.9191  |  108121/5634041 |
|   L2   |  1.8329  |  103268/5634041 |
| EFCalo |  1.5639  |  88112/5634041  |
|  HLT   |  0.0466  |   2627/5634041  |
+--------+----------+-----------------+


In [21]:
eff_data17_JF17.save()

In [22]:
del data17_df

## Emulate 2018:

In [23]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [24]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.35s/it]


In [25]:
data18_df.shape

(41873973, 111)

In [26]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v12.root')

2022-03-22 18:26:42,527 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v12.root


In [27]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:51<00:00, 22.38s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5751  | 30524649/32620495 |
|   L2   | 93.3469  | 30450228/32620495 |
| EFCalo | 92.0106  | 30014311/32620495 |
|  HLT   | 90.5622  | 29541825/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.36s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1980  | 21961165/23313826 |
|   L2   | 94.1035  | 21939128/23313826 |
| EFCalo | 93.3041  | 21752747/23313826 |
|  HLT   | 87.5989  | 20422665/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3658  | 1345279/1381675 |
|   L2   | 97.2808  | 1344105/1381675 |
| EFCalo | 95.1917  | 1315240/1381675 |
|  HLT   | 92.2068  | 1273998/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.36it/s]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.0559  | 59215/60389  |
|   L2   | 97.9649  | 59160/60389  |
| EFCalo | 96.7809  | 58445/60389  |
|  HLT   | 94.9395  | 57333/60389  |
+--------+----------+--------------+


In [28]:
eff_data18_Zee.save()

In [29]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v12.root')

2022-03-22 18:30:48,176 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v12.root


In [30]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v12'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.55s/it]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.6081  |  220827/8467098 |
|   L2   |  2.5369  |  214800/8467098 |
| EFCalo |  2.3475  |  198764/8467098 |
|  HLT   |  0.6824  |  57779/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.1768  |  89777/7628804  |
|   L2   |  1.1295  |  86170/7628804  |
| EFCalo |  0.9912  |  75618/7628804  |
|  HLT   |  0.0462  |   3527/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.73s/it]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  1.1612  |  76195/6561985  |
|   L2   |  1.1063  |  72595/6561985  |
| EFCalo |  0.9417  |  61794/6561985  |
|  HLT   |  0.0568  |   3729/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.06s/it]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.4843  |  118662/4776518 |
|   L2   |  2.3928  |  114293/4776518 |
| EFCalo |  2.1357  |  102011/4776518 |
|  HLT   |  0.3559  |  16998/4776518  |
+--------+----------+-----------------+


In [31]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [32]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [33]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 37.29it/s]


In [34]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v12.root')

2022-03-22 18:32:01,873 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v12.root


In [35]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v12_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v12'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 15.99it/s]


HLT_e17_lhvloose_nod0_ringer_v12_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 98.9316  | 77779/78619  |
|   L2   | 98.9214  | 77771/78619  |
| EFCalo | 77.6950  | 61083/78619  |
|  HLT   | 76.6189  | 60237/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.27it/s]


HLT_e28_lhtight_nod0_ringer_v12_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 94.3506  | 68975/73105  |
|   L2   | 94.3451  | 68971/73105  |
| EFCalo | 75.1795  | 54960/73105  |
|  HLT   | 72.4068  | 52933/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.97it/s]


HLT_e60_lhmedium_nod0_ringer_v12_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 95.7478  | 70794/73938  |
|   L2   | 95.7424  | 70790/73938  |
| EFCalo | 75.6228  | 55914/73938  |
|  HLT   | 73.8362  | 54593/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.04it/s]

HLT_e140_lhloose_nod0_ringer_v12
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 98.9140  | 67217/67955  |
|   L2   | 98.9096  | 67214/67955  |
| EFCalo | 77.0642  | 52369/67955  |
|  HLT   | 75.6089  | 51380/67955  |
+--------+----------+--------------+


In [36]:
eff_mc16.save()

## 